In [1]:
from collections import deque

numpad = [[7,8,9],
          [4,5,6],
          [1,2,3],
          ['',0,'A']]

numpad_move = [[[[0,0,0],
                [0,0,0],
                [0,0,0],
                [0,0,0]] for _ in range(3)] for _ in range(4)]

def map_num(n):
    if n == '7':
        return (0,0)
    elif n == '8':
        return (0,1)
    elif n == '9':
        return (0,2)
    elif n == '4':
        return (1,0)
    elif n == '5':
        return (1,1)
    elif n == '6':
        return (1,2)
    elif n == '1':
        return (2,0)
    elif n == '2':
        return (2,1)
    elif n == '3':
        return (2,2)
    elif n == '0':
        return (3,1)
    elif n == 'A':
        return (3,2)

dirs = [(-1,0), (1,0), (0,1), (0,-1)]
dirs_map = {(-1,0):'^', (1,0):'v', (0,1):'>', (0,-1):'<'}
inv_dirs_map = {dirs_map[key]:key for key in dirs_map}

for j in range(4):
    for i in range(3):
        queue = deque([(j,i,'')])
        ref = numpad_move[j][i]

        while queue:
            (nj, ni, his) = queue.popleft()
            ref[nj][ni] = his
            for dir in dirs:
                nnj = nj + dir[0]
                nni = ni + dir[1]
                if 0<=nnj<4 and 0<=nni<3 and ref[nnj][nni]==0 and numpad[nnj][nni]!='':
                    queue.append((nnj,nni,his + dirs_map[dir]))

In [2]:
remote = [['','^','A'],
          ['<','v','>']]

remote_move = [[[[0,0,0],
               [0,0,0]] for _ in range(3)] for _ in range(2)]

def map_remote(n):
    if n == '^':
        return (0,1)
    elif n == 'A':
        return (0,2)
    elif n == '<':
        return (1,0)
    elif n == 'v':
        return (1,1)
    elif n == '>':
        return (1,2)
    
for j in range(2):
    for i in range(3):
        queue = deque([(j,i,'')])
        ref = remote_move[j][i]

        while queue:
            (nj, ni, his) = queue.popleft()
            ref[nj][ni] = his
            for dir in dirs:
                nnj = nj + dir[0]
                nni = ni + dir[1]
                if 0<=nnj<2 and 0<=nni<3 and ref[nnj][nni]==0 and remote[nnj][nni]!='':
                    queue.append((nnj,nni,his+dirs_map[dir]))

In [15]:
from itertools import permutations

def cost_seq(seq):
    seq = seq + 'A'
    cost = 0
    curr = map_remote('A')
    for i in seq:
        cost += abs(curr[0] - map_remote(i)[0]) + abs(curr[1] - map_remote(i)[1])
        curr = map_remote(i)
    return cost

cache = {}

def permute(seq, start, map):
    if (seq, start) in cache:
        return cache[(seq, start)]
    else:
        min_cost = float('inf')
        poss = []
        for s in [''.join(i) for i in permutations(seq)]:
            curr = start
            for i in s:
                curr = (curr[0] + inv_dirs_map[i][0], curr[1] + inv_dirs_map[i][1])
                if map[curr[0]][curr[1]] == '':
                    break
            else:
                if min_cost > cost_seq(s):
                    min_cost = cost_seq(s)
                    poss = [s + 'A']
                elif min_cost == cost_seq(s):
                    poss.append(s + 'A')
        cache[(seq, start)] = poss
    return cache[(seq, start)]

# cost_seq('v<')

In [4]:
cache

{}

In [16]:
def move(map, start, end):
    return map[start[0]][start[1]][end[0]][end[1]]

with open("in21.txt", "r") as f:
    tar_nums = f.readlines()
    tar_nums = [i[:-1] for i in tar_nums]

depth_max = 4
def keypad_rob(perm, depth):
    # print(depth)
    global depth_max
    best_cost = float('inf')
    best_seq = ''
    for tar_rem in perm:
        p_rem = map_remote('A')
        curr_seq = ''
        for tr in (tar_rem):
            np_rem = map_remote(tr)
            if depth != depth_max:
                curr_seq += keypad_rob(permute(move(remote_move, p_rem, np_rem), p_rem, remote), depth + 1)
            else:
                curr_seq += move(remote_move, p_rem, np_rem) + 'A'
            p_rem = np_rem
        if len(curr_seq) < best_cost:
            best_cost = len(curr_seq)
            best_seq = curr_seq
    return best_seq

final_res = 0
for tar_num in tar_nums:
    p_num = map_num('A')
    res = ''
    for tn in tar_num:
        np_num = map_num(tn)
        res += keypad_rob(permute(move(numpad_move, p_num, np_num), p_num, numpad), 1)
        p_num = np_num

    # print(len(res))
    # print(res)
    final_res += int(tar_num[:-1])*len(res)
print(final_res)

757754


In [24]:
def move(map, start, end):
    return map[start[0]][start[1]][end[0]][end[1]]

with open("in21.txt", "r") as f:
    tar_nums = f.readlines()
    tar_nums = [i[:-1] for i in tar_nums]

depth_max = 2

# cached = {i:{'<':0,
#              '^':0,
#              '>':0,
#              'v':0,
#              'A':0} for i in range(1, depth_max+1)}

cached = {i:{} for i in range(1, depth_max+1)}

def keypad_rob(perm, depth):
    # print(depth)
    global depth_max
    best_cost = float('inf')
    for tar_rem in perm:
        p_rem = map_remote('A')
        curr_seq_cost = 0
        for tr in (tar_rem):
            np_rem = map_remote(tr)
            temp = (p_rem, np_rem)
            if temp in cached[depth]:
                curr_seq_cost += cached[depth][temp]
                continue
            if depth != depth_max:
                cached[depth][temp] = keypad_rob(permute(move(remote_move, p_rem, np_rem), p_rem, remote), depth + 1)
                curr_seq_cost += cached[depth][temp]
            else:
                cached[depth][temp] = len(move(remote_move, p_rem, np_rem) + 'A')
                curr_seq_cost += cached[depth][temp]
            p_rem = np_rem
        if curr_seq_cost < best_cost:
            best_cost = curr_seq_cost
    return best_cost

final_res = 0
for tar_num in tar_nums:
    p_num = map_num('A')
    res_cost = 0
    for tn in tar_num:
        np_num = map_num(tn)
        res_cost += keypad_rob(permute(move(numpad_move, p_num, np_num), p_num, numpad), 1)
        p_num = np_num

    # print(len(res))
    print(res_cost)
    final_res += int(tar_num[:-1])*res_cost
print(final_res)

1862481386
2184407992
2636337358
2058844484
2636337358
4604641122822


In [41]:
move(numpad_move, [0,2], [1,0])

'<<v'

In [ ]:
<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A
<v<AA>A>^AAvA<^A>AAvA^A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A

In [27]:

final_res = 0
for tar_num in tar_nums:
    p_num = map_num('A')
    p_rem1 = map_remote('A')
    p_rem2 = map_remote('A')

    res = ''

    for tn in tar_num:
        np_num = map_num(tn)
        best_cost = float('inf')
        best_seq = ''
        for tar_rem1 in permute(move(numpad_move, p_num, np_num), p_num, numpad):
            curr_seq = ''
            for tr1 in (tar_rem1):
                np_rem1 = map_remote(tr1)
                best_cost1 = float('inf')
                best_seq1 = ''
                for tar_rem2 in permute(move(remote_move, p_rem1, np_rem1), p_rem1, remote):
                    curr_seq1 = ''
                    for tr2 in (tar_rem2):
                        np_rem2 = map_remote(tr2)
                        tar_rem3 = move(remote_move, p_rem2, np_rem2) + 'A'
                        curr_seq1 += (tar_rem3)
                        p_rem2 = np_rem2

                    if len(curr_seq1) < best_cost1:
                        best_seq1 = curr_seq1
                        best_cost1 = len(curr_seq1)
                curr_seq += best_seq1
                p_rem1 = np_rem1
            if len(curr_seq) < best_cost:
                best_cost = len(curr_seq)
                best_seq = curr_seq
        res+= best_seq
        p_num = np_num

    print(len(res))
    print(res)
    final_res += int(tar_num[:-1])*len(res)
print(final_res)

74
<v<A>>^A<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
72
<vA<AA>>^AvA<^A>AAvA^A<vA<AA>>^AvAA<^A>A<vA>^A<v<A>>^AAvA<^A>A<vA>^A<A>A
66
<v<A>>^AAA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAAvA<^A>A
74
<vA<AA>>^AvA<^A>AAvA^A<vA>^A<v<A>^A>AvA^A<v<A>A>^AvA<^A>A<v<A>A>^AAvA<^A>A
68
<vA<AA>>^AvA<^A>AAvA^A<v<A>>^AvA^A<v<A>A>^AAvA<^A>A<v<A>A>^AvA^A<A>A
184180


In [ ]:
def seq_cost(seq):
    start = map_remote("A")
    curr_cost = 0
    for m in seq:
        m = map_remote(m)
        curr_cost += abs(m[0] - start[0]) + abs(m[1] - start[1])
        start = m
    return curr_cost

def minimize(map, seq, inv_map):
    best_cost = float('inf')
    best_seq = ''
    for perm in permutations(seq):
        start = map("A")
        curr_cost = 0
        for m in perm:
            m_copy = m
            m = map(m)
            print(m)
            curr_cost += abs(m[0] - start[0]) + abs(m[1] - start[1])
            start = m
        if curr_cost < best_cost:
            curr = map("A")
            for m in perm:
                curr = (curr[0] + inv_dirs_map[m_copy][0], curr[1] + inv_dirs_map[m_copy][1])
                if inv_map[curr[0]][curr[1]] == '':
                    continue
            best_cost = curr_cost
            best_seq = ''.join(perm)
    return best_seq

tar_num = '029A'
p_num = map_num('A')
p_rem1 = map_remote('A')
p_rem2 = map_remote('A')

res = ''

def move(map, start, end):
    return map[start[0]][start[1]][end[0]][end[1]]

for tn in tar_num:
    np_num = map_num(tn)
    tar_rem1 = minimize(map_remote, move(numpad_move, p_num, np_num), numpad) + 'A'
    for tr1 in (tar_rem1):
        np_rem1 = map_remote(tr1)
        tar_rem2 = minimize(map_remote, move(remote_move, p_rem1, np_rem1), remote) + 'A'
        for tr2 in (tar_rem2):
            np_rem2 = map_remote(tr2)
            tar_rem3 = minimize(map_remote, move(remote_move, p_rem2, np_rem2), remote) + 'A'
            res += (tar_rem3)
            # res += (tar_rem3 + 'A' +f"[{p_rem2}][{np_rem2}]")
            p_rem2 = np_rem2
        # res += f"[{p_rem1}][{np_rem1}]"
        p_rem1 = np_rem1
    # res += f"[{p_rem1}][{np_rem1}]"
    p_num = np_num
print(len(res))
print(res)

(1, 0)
(1, 0)
(1, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 1)


IndexError: list index out of range